# Import

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import lxml
import sys
import time
import re

# chromedriver 위치 설정

In [2]:
chromedriver_dir = '/Users/mongkyo/projects/mango-plate/Crawling/chromedriver'
# options = webdriver.ChromeOptions()
# options.add_argument('headless')
# options.add_argument('window-size=1920x1080')
# options.add_argument("disable-gpu")
# driver = webdriver.Chrome(chormedriver_dir, chrome_option=options)

# for문을 사용하여 성수역 검색결과 1페이지 식당 들어갔다 나오기

### 식당 별 for문 돌면서 딕셔너리에 값 가져오기. (리스트 안에 디셔너리들을 넣음)

In [3]:
store_info = []
info = dict()

In [ ]:
driver = webdriver.Chrome(chromedriver_dir)
driver.get('https://www.mangoplate.com/')
all_iframes = driver.find_elements_by_tag_name("iframe")
if len(all_iframes) > 0:
    print("Ad Found\n")
    driver.execute_script("""
        var elems = document.getElementsByTagName("iframe"); 
        for(var i = 0, max = elems.length; i < max; i++)
             {
                 elems[i].hidden=true;
             }
                          """)
    print('Total Ads: ' + str(len(all_iframes)))
else:
    print('No frames found')
driver.get('https://www.mangoplate.com/')
elem = driver.find_element_by_name("main-search")
elem.send_keys("성수역")
elem.send_keys(Keys.RETURN)
for page in range(1, 2+1):
    if page == 1:
        pass
    else:
        driver.find_element_by_xpath(f'/html/body/main/article/div[2]/div/div/section/div[4]/p/a[{page}]').click()
        time.sleep(3)
    for li in range(1,10+1):
        for div in range(1,2+1):
            driver.find_element_by_xpath(f'/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[{li}]/div[{div}]/figure/a').click()
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            title = soup.find("h1", {"class": "restaurant_name"})
    #         rating = soup.find("strong", {"class": "rate-point"})
            info = dict()
            info['name'] = title.get_text()
    #         info['평점'] = rating.get_text()
            table = soup.find("tbody")
            for thtd in table.find_all("tr"):
                if thtd.th.get_text() != "메뉴":
                    temp = thtd.th.get_text().replace(' ', '')
                    if temp.replace('\n', '') == '주소':
                        info['address'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '전화번호':
                        info['phone'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '음식종류':
                        info['food_type'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '가격대':
                        info['pricee_level'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '주차':
                        info['parking'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '영업시간':
                        info['Business_hour'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '쉬는시간':
                        info['break_time'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '마지막주문':
                        info['last_order'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '휴일':
                        info['holiday'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '사이트':
                        info['website'] = thtd.td.get_text().replace('\n', '')
                    elif temp.replace('\n', '') == '유튜브':
                        info['youtube'] = thtd.td.get_text().replace('\n', '')
                else:
                    info[thtd.th.get_text()] = thtd.td.get_text().replace('\n', '')
            time.sleep(1)
            map_tag = soup.select_one('body > main > article > div.column-wrapper > div.side-wrap > div > div.map-container.daum_map_wrap > img')['src']
            p = re.compile(r'center=(?P<longitude>[\d\.]+),(?P<latitude>[\d\.]+)')
            k= re.search(p, map_tag)
            info['longitude'] = k.group('longitude')
            info['latitude'] = k.group('latitude')
            store_info.append(info)
            driver.execute_script("window.history.go(-1)")

Ad Found

Total Ads: 3


### pagenation

In [16]:
store_info

[{'name': '소문난성수감자탕',
  'address': '서울시 성동구 성수동2가 315-100',
  'phone': '02-465-6580',
  'food_type': '탕 / 찌개 / 전골',
  'pricee_level': '만원-2만원',
  'parking': '무료주차 가능  ',
  'Business_hour': '00:00 - 24:00',
  '메뉴': '감자국식사7,500원감자탕 (소)24,000원감자탕 (중)27,000원감자탕 (대)36,000원'},
 {'name': '성수족발',
  'address': '서울시 성동구 성수동2가 289-273',
  'phone': '02-464-0425',
  'food_type': '고기 요리',
  'pricee_level': '만원-2만원',
  'parking': '주차공간없음  ',
  'Business_hour': '12:00 - 22:00',
  '메뉴': '족발 (중)35,000원족발 (대)40,000원족발 (특대)45,000원'},
 {'name': '자그마치',
  'address': '서울시 성동구 성수동2가 317-12',
  'phone': '070-4409-7700',
  'food_type': '카페 / 디저트',
  'pricee_level': '만원 미만',
  'parking': '주차공간없음  ',
  'Business_hour': '11:00 - 23:00',
  'last_order': '22:30',
  '메뉴': '아메리카노5,000원카페라떼5,500원플랫화이트5,000원'},
 {'name': '어니언',
  'address': '서울시 성동구 성수동2가 277-135',
  'phone': '070-7816-2710',
  'food_type': '카페 / 디저트',
  'pricee_level': '만원 미만',
  'parking': '주차공간없음 ',
  'Business_hour': '월-금: 08:00 - 22:00 토-일: 10:00 -

In [6]:
soup

<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml"><head><style type="text/css">@charset "UTF-8";[ng\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide:not(.ng-hide-animate){display:none !important;}ng\:form{display:block;}</style>
<!-- HTML Default Meta -->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="힙스터스 - 왕십리/성동 칵테일 / 와인 | 맛집검색 망고플레이트" name="subject"/>
<meta content="560명이 방문하고 싶어하는 왕십리/성동 식당 힙스터스의 리뷰, 영업시간, 그리고 비슷한 레스토랑을 알아보세요." name="description"/>
<meta content="" name="keywords"/>
<meta content="https://mp-seoul-image-production-s3.mangoplate.com/354157/550285_1534483734942_45688" name="image"/>
<!-- Follow RDFa Meta-->
<meta content="https://www.mangoplate.com/restaurants/FMBl5cwojeB6" pr

In [9]:
map_static = soup.select_one('body > main > article > div.column-wrapper > div.side-wrap > div > div.map-container.daum_map_wrap > img')

In [10]:
map_static

<img alt="naver_map_image" class="naver_static_map" src="https://openapi.naver.com/v1/map/staticmap.bin?clientId=_arHno93XUB1pXRNOA9w&amp;url=https%3A%2F%2Fwww.mangoplate.com%2Frestaurants%2FFMBl5cwojeB6&amp;center=127.056013,37.543035&amp;level=10&amp;w=400&amp;h=328&amp;baselayer=default"/>

In [13]:
map_tag = soup_tag.select_one('body > main > article > div.column-wrapper > div.side-wrap > div > div.map-container.daum_map_wrap > img')['src']
map_tag

'https://openapi.naver.com/v1/map/staticmap.bin?clientId=_arHno93XUB1pXRNOA9w&url=https%3A%2F%2Fwww.mangoplate.com%2Frestaurants%2FE8Qy6R_OJXhQ&center=127.055109,37.5431613&level=10&w=400&h=328&baselayer=default'

In [14]:
map_tag = soup.select_one('body > main > article > div.column-wrapper > div.side-wrap > div > div.map-container.daum_map_wrap > img')
soup_tag = BeautifulSoup(html, 'lxml')
map_tag = soup_tag.select_one('body > main > article > div.column-wrapper > div.side-wrap > div > div.map-container.daum_map_wrap > img')['src']
map_tag

'https://openapi.naver.com/v1/map/staticmap.bin?clientId=_arHno93XUB1pXRNOA9w&url=https%3A%2F%2Fwww.mangoplate.com%2Frestaurants%2FE8Qy6R_OJXhQ&center=127.055109,37.5431613&level=10&w=400&h=328&baselayer=default'

In [20]:
map_tag = soup.select_one('body > main > article > div.column-wrapper > div.side-wrap > div > div.map-container.daum_map_wrap > img')['src']
map_tag

'https://openapi.naver.com/v1/map/staticmap.bin?clientId=_arHno93XUB1pXRNOA9w&url=https%3A%2F%2Fwww.mangoplate.com%2Frestaurants%2FE8Qy6R_OJXhQ&center=127.055109,37.5431613&level=10&w=400&h=328&baselayer=default'

In [21]:
p = re.compile(r'center=(?P<longitude>[\d\.]+),(?P<latitude>[\d\.]+)')
# k = re.search(', map_tag)

In [22]:
k= re.search(p, map_tag)
print(k.group('longitude'), k.group('latitude'))

127.055109 37.5431613


In [23]:
info['longitude'] = k.group('longitude')
info['latitude'] = k.group('latitude')

In [12]:
map_tag = soup.select_one('body > main > article > div.column-wrapper > div.side-wrap > div > div.map-container.daum_map_wrap > img')['src']
p = re.compile(r'center=(?P<longitude>[\d\.]+),(?P<latitude>[\d\.]+)')
k= re.search(p, map_tag)
info['longitude'] = k.group('longitude')
info['latitude'] = k.group('latitude')
info

127.056013 37.543035


{'name': '힙스터스',
 'address': '서울시 성동구 성수동2가 315-17',
 'phone': '010-9181-9077',
 'food_type': '칵테일 / 와인',
 'pricee_level': '만원-2만원',
 'parking': '주차공간없음 ',
 'Business_hour': '월-목: 18:30 - 00:30 금-토: 18:30 - 02:30',
 'holiday': '일',
 'longitude': '127.056013',
 'latitude': '37.543035'}

In [24]:
info

{'name': '핑거팁스',
 'address': '서울시 성동구 성수동2가 315-36',
 'phone': '070-4232-0805',
 'food_type': '브런치 / 버거 / 샌드위치',
 'pricee_level': '만원 미만',
 'parking': '주차공간없음 ',
 'Business_hour': '화-금: 11:00 - 22:00 토-일: 11:00 - 21:00',
 'break_time': '월-금: 15:20 - 17:30',
 'holiday': '월',
 'longitude': '127.055109',
 'latitude': '37.5431613'}

In [25]:
soup = BeautifulSoup(html, 'html.parser')
title = soup.find("h1", {"class": "restaurant_name"})
#         rating = soup.find("strong", {"class": "rate-point"})
info = dict()
info['name'] = title.get_text()
#         info['평점'] = rating.get_text()
table = soup.find("tbody")
for thtd in table.find_all("tr"):
    if thtd.th.get_text() != "메뉴":
        temp = thtd.th.get_text().replace(' ', '')
        if temp.replace('\n', '') == '주소':
            info['address'] = thtd.td.get_text().replace('\n', '')
        elif temp.replace('\n', '') == '전화번호':
            info['phone'] = thtd.td.get_text().replace('\n', '')
        elif temp.replace('\n', '') == '음식종류':
            info['food_type'] = thtd.td.get_text().replace('\n', '')
    else:
        info[thtd.th.get_text()] = thtd.td.get_text().replace('\n', '')

In [68]:
info

{'name': '미야꼬',
 'address': '서울시 성동구 성수동2가 277-20',
 'phone': '02-462-8341',
 'food_type': '정통 일식 / 일반 일식'}